In [48]:
import requests               
import pandas as pd          
from scrapy import Selector 
import pprint 
from tqdm import tqdm
import spacy

In [ ]:
bbclink = 'https://www.bbcgoodfood.com/search?q='
response = requests.get(bbclink)
sel = Selector(text=response.text)
links = []

recipecards = sel.css('main div.search-results div.card__section.card__content a ::attr(href)').getall()
links.extend(recipecards)  
prefix = "https://www.bbcgoodfood.com/recipes/"
final_links = [prefix + item for item in links] 
print(final_links)

In [17]:
base_url = 'https://www.bbcgoodfood.com/search?q=&limit=1000&page='

links = []
for i in tqdm(range(1, 11)):
    response = requests.get(base_url + str(i))
    sel = Selector(text = response.text)
    recipecards = sel.css('main div.search-results div.card__section.card__content a ::attr(href)').getall()
    links.extend(recipecards)  

prefix = "https://www.bbcgoodfood.com/recipes/"
final_links = [prefix + item for item in links] 

len(final_links)

100%|██████████| 10/10 [01:33<00:00,  9.38s/it]


10011

### Create a DataFrame containing the nutritional data of all the recipes on BBCGoodFood 

In [45]:
session = requests.Session()

def get_nutrition(url):
    response2 = session.get(url)
    sel2 = Selector(text=response2.text)
    table = sel2.css('table.key-value-blocks.hidden-print.mt-xxs')
    bbc = {}
    bbc['name'] = sel2.css('h1.heading-1 ::text').get()
    bbc['calories'] = table.css('td.key-value-blocks__value ::text').get()
    bbc['salt'] = table.xpath('.//*[contains(text(), "salt")]/..').css('td.key-value-blocks__value ::text').get()
    bbc['fat'] = table.xpath('.//*[contains(text(), "fat")]/..').css('td.key-value-blocks__value ::text').get()
    return bbc

bbc = [get_nutrition(url) for url in tqdm(final_links)]

df_bbc = pd.DataFrame(bbc)
df_bbc.head()

100%|██████████| 10011/10011 [2:44:43<00:00,  1.01it/s]


,name,calories,salt,fat
0,Chicken & chorizo jambalaya,445,1.2,10
1,Lemon drizzle cake,399,0.3,21
2,Chilli con carne recipe,387,2.32,17
3,Best ever chocolate brownies recipe,150,0.1,9
4,Creamy courgette lasagne,405,1.36,21


### Save the data as a CSV file 

In [47]:
df_bbc.to_csv('../data/bbc_data.csv', index=False)

In [50]:
df_filtered = pd.read_csv('../data/filtered_posts.csv')
df_filtered.head()

,title,created_utc,ups,downs,upvote_ratio,score,num_comments,is_original_content,permalink,url
0,Prawn Katsu Baos,2024-01-17 21:56:28,256,False,0.95,256,9,False,https://reddit.com/r/recipes/comments/1998zka/...,https://i.redd.it/q81uyef4o2dc1.jpeg
1,Cinnamon Rolls,2024-01-05 17:45:11,251,False,0.96,251,21,False,https://reddit.com/r/recipes/comments/18zcqmd/...,https://i.redd.it/7uef78dbsnac1.jpeg
2,Bacon Jalapeño Sweet Potato Hash,2024-01-03 23:42:20,111,False,0.97,111,9,False,https://reddit.com/r/recipes/comments/18xxyl1/...,https://i.redd.it/jcbdya99abac1.jpeg
3,Mushroom-Taleggio Risotto,2023-12-31 19:25:22,174,False,0.98,174,6,False,https://reddit.com/r/recipes/comments/18vf164/...,https://i.redd.it/qc5akriilo9c1.jpeg
4,Cinnamon Oatmeal Chocolate Chip Cookies (Recipe),2023-12-31 13:19:31,206,False,0.95,206,16,False,https://reddit.com/r/recipes/comments/18v7m3w/...,https://i.redd.it/aki9a36yrm9c1.jpeg


In [59]:
df_bbc.tail()

,name,calories,salt,fat
10006,Cinnamon apple & raisin porridge,206,0.1,4
10007,Grilled salmon teriyaki with cucumber salad,340,4.23,18
10008,Persimmon & white chocolate tart with pistachios,934,0.8,67
10009,Christmas ceviche with guacamole,131,1.4,10
10010,"Thai prawn, squash & pineapple curry",292,2.4,17


In [57]:
nlp = spacy.load('en_core_web_lg')
similarity_threshold = 0.7

df_reddit = df_filtered.copy()

# Columns to merge from df_B to df_A when there is a match
merge_columns = ['calories', 'salt', 'fat']

# Iterate through each pair of titles and merge columns if similarity is above the threshold
for i in range(len(df_reddit)):
    title_A = df_reddit.iloc[i]['title']
    doc_A = nlp(title_A)

    for j in range(len(df_bbc)):
        title_B = df_bbc.iloc[j]['name']
        doc_B = nlp(title_B)

        if doc_A.similarity(doc_B) > similarity_threshold:
            for col in merge_columns:
                df_reddit.at[i, f'{col}_from_bbc'] = df_bbc.iloc[j][col]

df_reddit.head()

/var/folders/m1/v5zskq_n3473s_qmdwjwwd7w0000gn/T/ipykernel_4063/2084550400.py:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if doc_A.similarity(doc_B) > similarity_threshold:


ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'NoneType'>

In [58]:
nlp = spacy.load('en_core_web_lg')
similarity_threshold = 0.7

df_reddit = df_filtered.copy()

def get_similarity(row_A, df_bbc):
    title_A = row_A['title']
    doc_A = nlp(title_A)
    
    # Apply the similarity calculation to the entire 'title' column in df_bbc
    df_bbc['similarity'] = df_bbc['name'].apply(lambda name_B: doc_A.similarity(nlp(name_B)))
    # Filter rows in df_bbc based on the threshold
    matches = df_bbc[df_bbc['similarity'] > similarity_threshold]
    
    # Merge columns from df_bbc to df_reddit if there are matches
    if not matches.empty:
        matched_columns = ['calories', 'salt', 'fat']
        for col in matched_columns:
            df_reddit[col + '_from_bbc'] = matches[col].values
    
    # Drop the temporary 'similarity' column from df_bbc
    df_bbc.drop(columns=['similarity'], inplace=True)

# Apply the custom function to each row in df_reddit
df_reddit.apply(get_similarity, df_bbc=df_bbc, axis=1)

df_reddit.head()

/var/folders/m1/v5zskq_n3473s_qmdwjwwd7w0000gn/T/ipykernel_4063/3254384372.py:11: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  df_bbc['similarity'] = df_bbc['name'].apply(lambda name_B: doc_A.similarity(nlp(name_B)))


ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'NoneType'>

In [62]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
df_reddit = df_filtered.copy()
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
similarity_threshold = 0.7

# Function to calculate cosine similarity between titles
def calculate_similarity(row_reddit, df_bbc):
    title_reddit = row_reddit['title']
    embedding_reddit = model.encode([title_reddit])[0].reshape(1, -1)

    similarities = []
    for _, row_bbc in df_bbc.iterrows():
        title_bbc = row_bbc['name']
        
        # Check if the embedding is not None before attempting to reshape
        embedding_bbc = model.encode([title_bbc])
        if embedding_bbc is not None and len(embedding_bbc) > 0:
            embedding_bbc = embedding_bbc[0].reshape(1, -1)
        else:
            # Handle the case where the embedding is None or empty
            continue

        similarity_score = cosine_similarity(embedding_reddit, embedding_bbc)[0][0]
        similarities.append(similarity_score)

    return similarities

# Columns to merge from df_bbc to df_reddit when there is a match
merge_columns = ['calories', 'salt', 'fat']

# Iterate through each row in df_reddit and calculate similarities with df_bbc
for i, row_reddit in df_reddit.iterrows():
    similarities = calculate_similarity(row_reddit, df_bbc)

    # Check if any similarity is above the threshold
    if any(similarity > similarity_threshold for similarity in similarities):
        # Find the index with the maximum similarity
        max_index = similarities.index(max(similarities))

        # Merge columns from df_bbc to df_reddit using values from the most similar title
        for col in merge_columns:
            df_reddit.at[i, f'{col}_from_df_bbc'] = df_bbc.iloc[max_index][col]

df_reddit.head()

TypeError: 'NoneType' object is not subscriptable

In [3]:
title1 = "Creamy Lemon-Basil Chicken Pasta"
title2 = "Chicken & bacon pasta"

from sentence_transformers import SentenceTransformer
sentences = [title1, title2]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)


[[-0.02184611  0.00591048  0.00187744 ... -0.00569096 -0.02545492
  -0.01958655]
 [-0.01549129  0.00366342  0.00322168 ...  0.0223089  -0.02564218
  -0.01029447]]


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
embedding1 = embeddings[0].reshape(1, -1)
embedding2 = embeddings[1].reshape(1, -1)

# Calculate cosine similarity
similarity_score = cosine_similarity(embedding1, embedding2)[0][0]
print(f"Cosine Similarity: {similarity_score}")

Cosine Similarity: 0.6690070629119873
